## Intel&reg; OSPRay Introduction
--------



## Learning Objectives

* Get familiar with the Intel® OSPRay renderer and API.
* Edit and compile code for the first lesson: the minimal ospTutorial sample.


***
## 1. What is Intel&reg; OSPRay?
Intel® OSPRay is a ray-tracing framework for high-quality visualization rendering. Intel OSPRay goals are to go beyond previous proof-of-concept ray tracing systems for visualization and offer a complete turn-key solution for existing production visualization software packages than can run efficiently on current hardware while offering modular device support for future hardware architectures. In particular, it introduces:

Small device-independent API for general but visualization-oriented ray tracing
Specific, CPU-oriented implementation of this API that provides an efficient visualization rendering engine for general-purpose CPU workstations and HPC resources

### 2. OSPRay Goals

The main challenge with ray tracing as a visualization rendering backend is that it does not easily map to existing rasterization oriented APIs—it requires new APIs that more generally target visualization applications, and then integration work for those applications to utilize the new APIs. More specifically, OSPRay is:

* **A library, not a visualization tool**. Rather than designing a brand new visualization package, OSPRay is a _library_ that many different visualization tools can then leverage.
* **A rendering solution for visualization tools**. Visualization tools are complex, often relying on middleware libraries (such as VTK). OSPRay does not replace or compete with such middleware, and focuses exclusively on the visualization pipeline’s rendering component. By broadening supported rendering primitives, shading models, data set sizes, etc OSPRay gives existing visualization tools’ analysis stages additional choices in what they can ask the rendering stage to do.
* **Focused on visualization rendering**. OSPRay emphasizes the rendering features needed by production scientific visualization - simple color-mapped geometry and palettes, and different renderers (primary, ambient occlusion and path tracing) that cater to a variety of needs. It does not aim for the photorealism of professional graphics, nor for game performance.
* **Focused on HPC visualization rendering**. Since “simple” problems are successfully handled by existing GPU-based approaches, we explicitly focus on problems that remain challenging for visualization applications, such as large data, volume rendering and advanced shading. In particular, we strive to enable effective and performant visualization across all kinds of HPC resources, even those that lack GPUs. We do not discourage GPU use for all problems, but offer an efficient alternative for platforms that do not have any, and, more generally, wish to advance ray tracing solutions for those problems that can benefit from its characteristics.
* **Focused on performance**. Though we do not have to achieve game-like frame rates, interactive data exploration requires performant rendering. Our implementation makes efficient use of threading, vectorization, and, if desired, node-parallelism; and leverages the most efficient ray tracing technologies available.

### 3. Intel OSPRay API
The Intel OSPRay API is a layer between visualization applications and low-level hardware resources. The figure below shows the Intel OSPRay API (the "layer cake") in relation to other hardware and software components commonly found in visualization applications. The API is designed to be platform independent - this implementation targets CPUs, but the API should also map to GPUs, integrated graphics, and so on. It is a low level of abstraction similar to that of OpenGL*, which is the abstraction level that modern visualization tools use for rendering. Similar to solutions in OpenGL and GPGPU, Intel OSPRay API focuses on a low-level data model.

<img src="assets/Figure3.png" style="width:50%"></img>

### 4. API Categories
The OSPRay API exposes the following categories of objects:

* **OSPFrameBuffers** hold the final result of a rendered frame. Information held can contain, but is not limited to, pixel colors, depth values, and accumulation information.
* **OSPData** are 1D data arrays, similar to “buffers” in a GPGPU context. In addition to the typical scalar and 2-, 3-, or 4-dimensional vector data, data arrays can also contain references to other actors (including to other data arrays), in device-abstract fashion.
* **OSPGeometry** contain geometric surface primitives such as triangles, spheres, cylinders, etc.
* **OSPVolumes** represent 3D scalar fields that can produce, for any 3D position, a scalar value that a volume renderer can sample.
* **OSPTransferFunctions** map scalars to RGBA colors.
* **OSPModels** are collections of geometries and volumes – the parent objects of the hierarchy. Time-varying data are vectors of OSP- Models.
* **OSPCameras** generate primary rays for renderers to compute on.
* **OSPRenderers** use cameras, models, etc, to render pixels. OSPRay defines two renderers: 
    * `scivis` renderer that combines many rendering techniques into a single renderer. In this renderer we focus on the needs of scientific visualization: we implement an OpenGL-like material model, with customizable contributions of transparency, shadows, ambient occlusion, and fully integrated volume rendering.
    * `pathtracer` renderer, a fully photo-realistic renderer that can be used for generating high-quality publication images, and that has since seen adoption even outside of scientific visualization.
* **OSPLights, OSPTextures, and OSPMaterials** specify additional inputs for rendering, lighting, shading, etc.

#### Commit transactions
An important aspect OSPRay is that parameters/data that affect any of the objects do not get passed to objects individually; instead, parameters are not visible at all to objects until they get explicitly committed to an object via a call to **ospCommit(object)**, at which time all previously additions or changes to parameters are visible at the same time. If a user wants to change the state of an existing object (e.g., to change the origin of an already existing camera) it is perfectly valid to do so, as long as the changed parameters are recommitted.

The commit semantics allow for batching up multiple small changes, and specifies exactly when changes to objects will occur. This is important to ensure performance and consistency for devices crossing a PCI* bus, or across a network. every commit.


***
### 5. First Lesson: ospTutorial

In order to get a sense for all of the various API components that need to come together to write an OSPRay visualizer we present ospTutorial - a minimal demonstration that exercises those components.

`ospTutorial` will create an image of two triangles, rendered with the pathtracer renderer. The image `firstFrame.png` shows the result after one call to `ospRenderFrame` – jagged edges and noise in the shadow can be seen. These are not incorrect renderings but are the result of the random nature of the ray tracing used to generate the image. In future lessons we will learn how to issue multiple calls to `ospRenderFrame` and converge the resulting image to display anti-aliased edges, for example.

#### 5.1 The Program Flow
The following is a general flow for the ospTutorial program. Though these steps are described for this program they generalize such that other OSPRay roughly implement these steps. Comments in the program follow along with these steps.

* Step 1 - Set up common objects to be used in the program
* Step 2 - Execute ospInit() to initialize the renderer
* Step 3 - Setup the camera into the scene
* Step 4 - Setup the scene
    * Step 4.1 Feeding the model/vertex data to OSPRay
    * Step 4.2 - Create a material 
    * Step 4.3 - Put the mesh into a model and apply material
    * Step 4.4 - Put the model into a group (collection of models)
    * Step 4.5 - Put the group into an instance (give the group a world transform)
    * Step 4.6 - Put the instance in the world
    * Step 4.7 - Create and setup light for Ambient Occlusion
* Step 5 - Setup of the renderer
    * Step 5.1 - Create renderer
    * Step 5.2 - Create and setup framebuffer
    * Step 5.3 - Render one frame
    * Step 5.4 - Access framebuffer and write its content as PNG file
* Step 6 - Shutdown system and clean up

In [ ]:
%%writefile src/ospTutorial.cpp


// system includes
#include <alloca.h>
#include <errno.h>
#include <stdint.h>
#include <stdio.h>

// ospray and math includes
#include "ospray/ospray_util.h"
#include "rkcommon/math/rkmath.h"
#include "rkcommon/math/vec.h"

// includes to STB for writing out a PNG image
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb/stb_image_write.h"

using namespace rkcommon::math;

// ################################################################################
// A simple function that uses the STB library to write out the framebuffer in
// PNG format.

void writePNG(const char *fileName, const vec2i &size, const uint32_t *pixel) {
  constexpr int nChannels{4};
  const int stride{nChannels * size.x};
  stbi_write_png(fileName, size.x, size.y, nChannels, pixel, stride);
}

// ################################################################################
// The entry point into the program that exercises the API

int main(int argc, const char **argv) {

  // ########## Step 1 - set up common objects to be used in the program

  // Define the width and height of the framebuffer
  // image size
  vec2i imgSize;
  imgSize.x = 1024;  // width
  imgSize.y = 768;   // height

  // camera
  // Placing the camera at the origin <0,0,0>
  float cam_pos[] = {0.f, 0.f, 0.f};

  // Orient the camera noting Y-up
  float cam_up[] = {0.f, 1.f, 0.f};

  // set the camera view direction
  float cam_view[] = {0.1f, 0.f, 1.f};

  // triangle mesh data
  // 4 vertices each with a XYZ position
  float vertex[] = {
    -1.0f, -1.0f, 3.0f, 
    -1.0f, 1.0f, 3.0f,
    1.0f, -1.0f, 3.0f, 
    0.1f,  0.1f, 0.3f};

  // 4 colors denoted by RGBA
  float color[] = {
    0.9f, 0.5f, 0.5f, 1.0f, 
    0.8f, 0.8f, 0.8f, 1.0f,
    0.8f, 0.8f, 0.8f, 1.0f, 
    0.5f, 0.9f, 0.5f, 1.0f};

  // index for the triangles
  int32_t index[] = {0, 1, 2, 1, 2, 3};

  // ########## Step 2 - execute ospInit() to initialize the renderer
    printf("initialize OSPRay...");

  // initialize OSPRay; OSPRay parses (and removes) its commandline parameters,
  OSPError init_error = ospInit(&argc, argv);
  if (init_error != OSP_NO_ERROR)
    return init_error;

  printf("done!\n");
  
  // ########## Step 3 - setup the camera into the scene
  printf("setting up camera...");

  // Feeding the camera data to OSPRay
  // create and setup camera
  OSPCamera camera = ospNewCamera("perspective");
  ospSetFloat(camera, "aspect", imgSize.x / (float)imgSize.y);
  ospSetParam(camera, "position", OSP_VEC3F, cam_pos);
  ospSetParam(camera, "direction", OSP_VEC3F, cam_view);
  ospSetParam(camera, "up", OSP_VEC3F, cam_up);
  ospCommit(camera);  // commit each object to indicate modifications are done

  printf("done!\n");

  // ########## Step 4 - setup the scene
  printf("setting up scene...");

  // #################### Step 4.1 Feeding the model/vertex data to OSPRay
  // create and setup model and mesh
  OSPGeometry mesh = ospNewGeometry("mesh");

  OSPData data = ospNewSharedData1D(vertex, OSP_VEC3F, 4);
  // alternatively with an OSPRay managed OSPData
  // OSPData managed = ospNewData1D(OSP_VEC3F, 4);
  // ospCopyData1D(data, managed, 0);

  ospCommit(data);
  ospSetObject(mesh, "vertex.position", data);
  ospRelease(data);  // we are done using this handle

  data = ospNewSharedData1D(color, OSP_VEC4F, 4);
  ospCommit(data);
  ospSetObject(mesh, "vertex.color", data);
  ospRelease(data);

  data = ospNewSharedData1D(index, OSP_VEC3UI, 2);
  ospCommit(data);
  ospSetObject(mesh, "index", data);
  ospRelease(data);

  ospCommit(mesh);

  // #################### Step 4.2 - create a material 
  OSPMaterial mat = ospNewMaterial("pathtracer", "obj");
  ospCommit(mat);


  // #################### Step 4.3 - put the mesh into a model and apply material
  OSPGeometricModel model = ospNewGeometricModel(mesh);
  ospSetObject(model, "material", mat);
  ospCommit(model);
  ospRelease(mesh);
  ospRelease(mat);

  // #################### Step 4.4 - put the model into a group (collection of models)
  OSPGroup group = ospNewGroup();
  ospSetObjectAsData(group, "geometry", OSP_GEOMETRIC_MODEL, model);
  ospCommit(group);
  ospRelease(model);

  /// #################### Step 4.5 - put the group into an instance (give the group a world transform)
  OSPInstance instance = ospNewInstance(group);
  ospCommit(instance);
  ospRelease(group);

  // #################### Step 4.6 - put the instance in the world
  OSPWorld world = ospNewWorld();
  ospSetObjectAsData(world, "instance", OSP_INSTANCE, instance);
  ospRelease(instance);

  // #################### Step 4.7 - create and setup light for Ambient Occlusion
  OSPLight light = ospNewLight("ambient");
  ospCommit(light);
  ospSetObjectAsData(world, "light", OSP_LIGHT, light);
  ospRelease(light);

  ospCommit(world);

  printf("done!\n");

  // print out world bounds
  OSPBounds worldBounds = ospGetBounds(world);
  printf("\nworld bounds: ({%f, %f, %f}, {%f, %f, %f}\n\n",
         worldBounds.lower[0], worldBounds.lower[1], worldBounds.lower[2],
         worldBounds.upper[0], worldBounds.upper[1], worldBounds.upper[2]);


  // ########## Step 5 - setup of the renderer
  printf("setting up renderer...");

  // #################### Step 5.1 - create renderer
  OSPRenderer renderer =
      ospNewRenderer("pathtracer");  // choose path tracing renderer

  // complete setup of renderer
  ospSetFloat(renderer, "backgroundColor", 1.0f);  // white, transparent
  ospCommit(renderer);

  // #################### Step 5.2 - create and setup framebuffer
  OSPFrameBuffer framebuffer =
      ospNewFrameBuffer(imgSize.x, imgSize.y, OSP_FB_SRGBA,
                        OSP_FB_COLOR | /*OSP_FB_DEPTH |*/ OSP_FB_ACCUM);
  ospResetAccumulation(framebuffer);

  printf("rendering initial frame to firstFrame.png...");

  // #################### Step 5.3 - render one frame
  ospRenderFrameBlocking(framebuffer, renderer, camera, world);

  // #################### Step 5.4 - access framebuffer and write its content as PNG file
  const uint32_t *fb = (uint32_t *)ospMapFrameBuffer(framebuffer, OSP_FB_COLOR);
  writePNG("firstFrame.png", imgSize, fb);
  ospUnmapFrameBuffer(fb, framebuffer);

  printf("done!\n");

  ospUnmapFrameBuffer(fb, framebuffer);

  printf("done!\n");

  // ########## Step 6 - shutdown system and clean up
  printf("\ncleaning up objects...");

  // final cleanups
  ospRelease(renderer);
  ospRelease(camera);
  ospRelease(framebuffer);
  ospRelease(world);

  printf("done!\n");

  ospShutdown();

  return 0;
}


### 6. Build the Code

In [ ]:
! ./q build.sh

### 7. Run the Program

In [ ]:
! ./q run.sh

### View the Resulting Image

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import figure
figure(num=None, figsize=(11, 8.5), dpi=72, facecolor='w', edgecolor='k')

img = mpimg.imread('firstFrame.png')
plt.axis('off')
imgplot = plt.imshow(img,aspect=None,interpolation='nearest')

***
## Summary

You have arrived at the end of this lesson. During this lesson, you:

* Learned about Intel OSPRay, a framework for ray-traced visualization rendering that advances ray tracing as a solution to some of today’s key rendering challenges in scientific visualization
* Got familiar with Intel OSPRay API for multi-core and many-core CPU architectures that runs well on hardware ranging from laptops to large-scale HPC resources
* Learned how to use the Intel OSPRay API to write a minimal viewer program for displaying mesh geometry

***
## Resources
* M. Pharr and G. Humphreys. Physically Based Rendering: From Theory to Implementation. Morgan Kaufman, 3rd edition, 2016.
* P. Shirley. Ray Tracing in One Weekend Series. Editors: Steve Hollasch, Trevor David Black. Version/Edition: v3.2.0. Date: 2020-07-18. URL (series): https://raytracing.github.io/
* I. Wald et al., "OSPRay - A CPU Ray Tracing Framework for Scientific Visualization," in IEEE Transactions on Visualization and Computer Graphics, vol. 23, no. 1, pp. 931-940, Jan. 2017, doi: 10.1109/TVCG.2016.2599041.

***
<html><body><span style="color:green"><h1>Next: ospExamples - an Example of Intel® OSPRay Techniques and Procedural Scenes</h1></span></body></html>

[Click Here](../03_ospExamples/ospExamples.ipynb)

<html><body><span style="color:green"><h1>Back: Overview</h1></span></body></html>

[Click Here](../../Overview.ipynb)